# TDC ADMET, Caco-2_Wang Submission

In [1]:
from typing import Tuple

import numpy as np
import pandas as pd

# cheminformatics
import rdkit.Chem
from rdkit.Chem import Descriptors
from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

# logging
import tqdm

# data preprocessing
import sklearn.impute
import sklearn.preprocessing

# modeling
import sklearn.ensemble
from sklearn.model_selection import ParameterGrid

# metrics
import sklearn.metrics

from tdc.single_pred import ADME


In [2]:
data = ADME(name = 'Caco2_Wang')
split = data.get_split()

Found local copy...
Loading...
Done!


In [3]:
from rdkit import Chem
from rdkit.Chem import MACCSkeys, Descriptors
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
import pandas as pd
import numpy as np
import tqdm

def add_descriptor_columns(
    data: pd.DataFrame,
    morgan_radius=2,
    morgan_nbits=1024,
    use_maccs=False,
    use_morgan=False,
    use_rdkit=True
) -> pd.DataFrame:
    """
    Calculate descriptors (MACCS, Morgan, RDKit) for each molecule in the dataframe.

    Parameters
    ----------
    data : pd.DataFrame
        Must contain 'Drug' (SMILES) and 'Y' (target) columns.
    morgan_radius : int
        Radius for Morgan fingerprints.
    morgan_nbits : int
        Number of bits for Morgan fingerprints.
    use_maccs, use_morgan, use_rdkit : bool
        Whether to include MACCS, Morgan, or RDKit descriptors.

    Returns
    -------
    df : pd.DataFrame
        DataFrame with selected descriptors + Y + Drug.
    """

    assert 'Drug' in data.columns, "'Drug' must be a column in the input DataFrame."
    assert 'Y' in data.columns, "'Y' must be a column in the input DataFrame."

    drugs = data['Drug']
    y = data['Y']

    fps = []
    print("Calculating descriptors...")

    rdkit_desc = [d for d in Descriptors._descList] if use_rdkit else []

    for smi, target in tqdm.tqdm(zip(drugs, y), total=len(drugs)):
        mol = Chem.MolFromSmiles(smi)

        row_parts = []

        if mol is None:
            if use_maccs:
                row_parts.append(np.zeros(167, dtype=int))
            if use_morgan:
                row_parts.append(np.zeros(morgan_nbits, dtype=int))
            if use_rdkit:
                row_parts.append(np.zeros(len(rdkit_desc), dtype=float))
        else:
            if use_maccs:
                maccs = np.array(MACCSkeys.GenMACCSKeys(mol))
                row_parts.append(maccs)

            if use_morgan:
                morgan = np.array(GetMorganFingerprintAsBitVect(mol, morgan_radius, nBits=morgan_nbits))
                row_parts.append(morgan)

            if use_rdkit:
                rdkit_vals = []
                for name, func in rdkit_desc:
                    try:
                        rdkit_vals.append(func(mol))
                    except Exception:
                        rdkit_vals.append(np.nan)
                rdkit_vals = np.array(rdkit_vals, dtype=float)
                row_parts.append(rdkit_vals)

        row = np.concatenate(row_parts + [[target]])
        fps.append(row)

    # Формируем имена колонок
    fp_columns = []
    if use_maccs:
        fp_columns += [f'maccs_{i}' for i in range(167)]
    if use_morgan:
        fp_columns += [f'morgan_{i}' for i in range(morgan_nbits)]
    if use_rdkit:
        fp_columns += [f'rdkit_{name}' for name, _ in rdkit_desc]

    fp_columns += ['Y']

    df = pd.DataFrame(fps, columns=fp_columns)
    df['Drug'] = drugs.values

    return df


def preprocess_data(
    data: pd.DataFrame, 
    imputer=sklearn.impute.SimpleImputer(missing_values=np.nan, strategy='mean'),
    fit_imputer=True,
    scaler_X=sklearn.preprocessing.RobustScaler(),
    scaler_y=sklearn.preprocessing.RobustScaler(),
    fit_scaler=True
):
    """
    Imputes missing values.
    Scales feature data.

    Returns a tuple X, y of scaled feature data and target data.
    """

    col_array = np.array(data.columns)

    # extract just the feature data
    X = data[col_array[~np.isin(col_array, ['Drug_ID', 'Drug', 'Y'])]].to_numpy()
    
    # extract the target data
    y = np.array(data['Y']).reshape(-1,1)
    
    # impute missing data
    if imputer is not None:
        if fit_imputer:
            X = imputer.fit_transform(X)
        else:
            X = imputer.transform(X)

    # scale the feature data
    if scaler_X is not None:
        if fit_scaler:
            X = scaler_X.fit_transform(X)
            y = scaler_y.fit_transform(y)
        else:
            X = scaler_X.transform(X)
            y = scaler_y.transform(y)



    return X, y, imputer, scaler_X, scaler_y

In [4]:
X_train, y_train, imputer, scaler_X, scaler_y = preprocess_data(
    add_descriptor_columns(split['train'])
)
X_val, y_val, _, _, _ = preprocess_data(
    add_descriptor_columns(split['valid']),
    imputer=imputer, fit_imputer=False,
    scaler_X=scaler_X, scaler_y=scaler_y,
    fit_scaler=False)


Calculating descriptors...


100%|██████████| 637/637 [00:08<00:00, 78.96it/s]


Calculating descriptors...


100%|██████████| 91/91 [00:01<00:00, 84.79it/s]


In [5]:
from tqdm import tqdm
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import ParameterGrid, RandomizedSearchCV
import numpy as np
import os
import json
import joblib

# Папка для сохранения модели и параметров
model_dir = "model/Cacao2"
os.makedirs(model_dir, exist_ok=True)

# ------------------ Stage 1: грубый поиск ------------------ #
params_grid_stage1 = {
    "hidden_layer_sizes": [(50,), (100,), (50,50), (100,50)],
    "activation": ["relu", "tanh"],
    "alpha": [10 ** i for i in range(-5, -1)],  # L2 penalty
    "learning_rate_init": [10 ** i for i in range(-4, -1)]
}

best_score = float('inf')
best_set = {}
best_model = None

for param_set in tqdm(ParameterGrid(params_grid_stage1), desc="Grid Search Stage 1"):
    model = MLPRegressor(max_iter=1000, random_state=42, **param_set)
    model.fit(X_train, y_train.ravel())

    y_val_pred_tmp = model.predict(X_val)
    score_MAE = mean_absolute_error(y_val, y_val_pred_tmp)

    if score_MAE < best_score:
        best_score = score_MAE
        best_set = param_set
        best_model = model

print("Лучшие параметры (Stage 1):", best_set)
print("MAE (Stage 1):", best_score)

# ------------------ Stage 2: уточнение ------------------ #
alpha_best = best_set["alpha"]
lr_best = best_set["learning_rate_init"]

params_grid_stage2 = {
    "hidden_layer_sizes": [best_set["hidden_layer_sizes"]],
    "activation": [best_set["activation"]],
    "alpha": [alpha_best * f for f in [0.5, 0.8, 1.0, 1.2, 2.0]],
    "learning_rate_init": [lr_best * f for f in [0.5, 0.8, 1.0, 1.2, 2.0]]
}

for param_set in tqdm(ParameterGrid(params_grid_stage2), desc="Grid Search Stage 2"):
    model = MLPRegressor(max_iter=1000, random_state=42, **param_set)
    model.fit(X_train, y_train.ravel())

    y_val_pred_tmp = model.predict(X_val)
    score_MAE = mean_absolute_error(y_val, y_val_pred_tmp)

    if score_MAE < best_score:
        best_score = score_MAE
        best_set = param_set
        best_model = model

print("Лучшие параметры (Stage 2):", best_set)
print("MAE (Stage 2):", best_score)

# ------------------ Stage 3: RandomizedSearchCV ------------------ #
param_distributions = {
    "hidden_layer_sizes": [(50,), (100,), (50,50), (100,50)],
    "activation": ["relu", "tanh"],
    "alpha": np.logspace(np.log10(best_set["alpha"] * 0.5), np.log10(best_set["alpha"] * 2), 50),
    "learning_rate_init": np.logspace(np.log10(best_set["learning_rate_init"] * 0.5),
                                     np.log10(best_set["learning_rate_init"] * 2), 50)
}

random_search = RandomizedSearchCV(
    MLPRegressor(max_iter=1000, random_state=42),
    param_distributions=param_distributions,
    n_iter=20,  # 20 случайных комбинаций
    scoring="neg_mean_absolute_error",
    cv=[(np.arange(len(X_train)), np.arange(len(X_val)))],  # имитация train/val split
    random_state=42,
    verbose=1,
    n_jobs=-1
)

random_search.fit(
    np.vstack((X_train, X_val)), 
    np.hstack((y_train.ravel(), y_val.ravel()))
)

best_model = random_search.best_estimator_
best_set = random_search.best_params_
best_score = -random_search.best_score_

print("Лучшие параметры (Stage 3 RandomizedSearchCV):", best_set)
print("MAE (Stage 3):", best_score)

# ------------------ Сохраняем модель и параметры ------------------ #
model_path = os.path.join(model_dir, "best_model_mlp.pkl")
joblib.dump(best_model, model_path)

params_path = os.path.join(model_dir, "best_params.json")
with open(params_path, "w") as f:
    json.dump({
        "best_score": best_score,
        "best_params": best_set
    }, f, indent=4)

print("✅ Лучшая модель сохранена в:", model_path)
print("✅ Параметры сохранены в:", params_path)


Grid Search Stage 1: 100%|██████████| 96/96 [00:26<00:00,  3.65it/s]


Лучшие параметры (Stage 1): {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (50,), 'learning_rate_init': 0.001}
MAE (Stage 1): 0.3109600658787877


Grid Search Stage 2: 100%|██████████| 25/25 [00:05<00:00,  4.27it/s]


Лучшие параметры (Stage 2): {'activation': 'tanh', 'alpha': 0.0002, 'hidden_layer_sizes': (50,), 'learning_rate_init': 0.0005}
MAE (Stage 2): 0.30556448362100064
Fitting 1 folds for each of 20 candidates, totalling 20 fits
Лучшие параметры (Stage 3 RandomizedSearchCV): {'learning_rate_init': 0.0008930016176563258, 'hidden_layer_sizes': (100, 50), 'alpha': 0.0002208179027347626, 'activation': 'tanh'}
MAE (Stage 3): 0.09870252538220474
✅ Лучшая модель сохранена в: model/Cacao2\best_model_mlp.pkl
✅ Параметры сохранены в: model/Cacao2\best_params.json


In [7]:
from tdc.benchmark_group import admet_group
from sklearn.neural_network import MLPRegressor
import numpy as np
import tqdm

group = admet_group(path='data/')
predictions_list = []

for seed in [1, 2, 3, 4, 5]:
    benchmark = group.get('Caco2_Wang') 
    predictions = {}
    name = benchmark['name']

    train_val, test = benchmark['train_val'], benchmark['test']

    print(f"Seed {seed}:")

    # ---------------- Предобработка ---------------- #
    X_train, y_train, imputer, scaler_X, scaler_y = preprocess_data(
        add_descriptor_columns(train_val)
    )
    X_test, y_test, _, _, _ = preprocess_data(
        add_descriptor_columns(test),
        imputer=imputer, fit_imputer=False,
        scaler_X=scaler_X,
        scaler_y=scaler_y,
        fit_scaler=False
    )

    # ---------------- MLPRegressor ---------------- #
    mlp_model = MLPRegressor(max_iter=1000, random_state=seed, **best_set)
    mlp_model.fit(X_train, y_train.ravel())

    # ---------------- Предсказания ---------------- #
    y_pred_test_scaled = mlp_model.predict(X_test)
    y_pred_test = scaler_y.inverse_transform(
        y_pred_test_scaled.reshape(-1, 1)
    ).reshape(-1)

    predictions[name] = y_pred_test
    predictions_list.append(predictions)

# ---------------- Оценка ---------------- #
results = group.evaluate_many(predictions_list)
print(results)


Found local copy...


Seed 1:
Calculating descriptors...


100%|██████████| 728/728 [00:08<00:00, 83.72it/s] 


Calculating descriptors...


100%|██████████| 182/182 [00:02<00:00, 77.92it/s]


...

Seed 5:
Calculating descriptors...


100%|██████████| 728/728 [00:08<00:00, 82.56it/s] 


Calculating descriptors...


100%|██████████| 182/182 [00:02<00:00, 77.90it/s]


{'caco2_wang': [0.41, 0.036]}
